In [1]:
import pandas as pd
import numpy as np

In [2]:
weather = pd.read_csv('datasets/seattleWeather/seattleWeather_1948-2017.csv')

In [3]:
weather

,DATE,PRCP,TMAX,TMIN,RAIN
0,1948-01-01,0.47,51,42,True
1,1948-01-02,0.59,45,36,True
2,1948-01-03,0.42,45,35,True
3,1948-01-04,0.31,45,34,True
4,1948-01-05,0.17,45,32,True
...,...,...,...,...,...
25546,2017-12-10,0.00,49,34,False
25547,2017-12-11,0.00,49,29,False
25548,2017-12-12,0.00,46,32,False
25549,2017-12-13,0.00,48,34,False


In [4]:
weather.drop("DATE", axis=1, inplace=True)

In [5]:
weather['RAIN'].value_counts()

False    14648
True     10900
Name: RAIN, dtype: int64

In [6]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25551 entries, 0 to 25550
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PRCP    25548 non-null  float64
 1   TMAX    25551 non-null  int64  
 2   TMIN    25551 non-null  int64  
 3   RAIN    25548 non-null  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 798.6+ KB


# Mark That label ie RAIN does not have all fields and also PRCP so we will drop those entris(For Rain)and fill the NA elements of PRCP with median value

In [7]:
a = weather.dropna(subset="RAIN")
print(a.shape, weather.shape)

(25548, 4) (25551, 4)


It is working fine now we will change the main dataframe

In [8]:
waether = weather.dropna(subset="RAIN", inplace=True)
weather.shape

(25548, 4)

Now filling the data of PRCP

In [9]:
median = weather['PRCP'].median()
weather['PRCP'].fillna(median, inplace=True)

In [10]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25548 entries, 0 to 25550
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PRCP    25548 non-null  float64
 1   TMAX    25548 non-null  int64  
 2   TMIN    25548 non-null  int64  
 3   RAIN    25548 non-null  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 998.0+ KB


# Splitting Data into train and Test 

In [11]:
def split_train_set(data, test_ratio):
    np.random.seed(42)
    shuffle = np.random.permutation(len(data))
    test_set_size = int(len(shuffled)*test_ratio)
    test_index = shuffled[:test_set_size]
    train_index = shuffled[test_set_size:]

In [12]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(weather, test_size=0.2, random_state=42)

# Splitting with Same Ratio of True/False

In [13]:
# from sklearn.model_selection import StratifiedShuffleSplit
# seperate = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
# for train_index, test_index in seperate.split(weather, weather['RAIN']):
#     strat_train_set = weather.loc[train_index]
#     strat_test_set = weather.loc[test_index]

# ABOVE CODE GIVING ERRORS

In [14]:
weather = train_set.copy()

# Finding Correlations (Not required in this Case as we need only True/False not a value)

In [15]:
matrix = weather.corr()
matrix
# If not True/False we would use
# matrix['RAIN'].sort_values(ascending=True/False)

C:\Users\rkesh\AppData\Local\Temp\ipykernel_16148\2448962165.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  matrix = weather.corr()


,PRCP,TMAX,TMIN
PRCP,1.000000,-0.226057,-0.064676
TMAX,-0.226057,1.000000,0.862095
TMIN,-0.064676,0.862095,1.000000


In [16]:
weather = train_set.drop("RAIN", axis=1)
weather_labels = train_set["RAIN"].copy()
weather_labels = weather_labels.astype("bool")
weather

,PRCP,TMAX,TMIN
11497,0.00,78,55
14118,0.00,88,65
1604,0.00,67,45
3693,0.03,52,39
4742,0.25,44,36
...,...,...,...
21578,0.00,40,28
5390,0.00,59,44
860,0.00,66,49
15795,0.00,61,41


In [17]:
# from sklearn.impute import SimpleImputer
# imputer = SimpleImputer(strategy="median")
# imputer.fit(weather)

# CANNOT USE Imputer due to DateField

In [18]:
weather.describe()

,PRCP,TMAX,TMIN
count,20438.000000,20438.000000,20438.000000
mean,0.106255,59.543302,44.524122
std,0.240267,12.792381,8.914362
min,0.000000,4.000000,0.000000
25%,0.000000,50.000000,38.000000
50%,0.000000,58.000000,45.000000
75%,0.100000,69.000000,52.000000
max,5.020000,103.000000,71.000000


In [19]:
from sklearn.linear_model import LogisticRegression

In [20]:
model = LogisticRegression()
model.fit(weather, weather_labels)

LogisticRegression()

In [22]:
model.predict([[0.00, 49, 34]])

c:\machine learning\venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


array([False])

# Testing The Model

In [23]:
test_feature = test_set.drop("RAIN", axis=1)
test_label = test_set["RAIN"].copy()
test_label = test_label.astype("bool")

In [24]:
final_predictions = model.predict(test_feature)